In [1]:
# !conda install -c conda-forge geopy --yes
# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge geocoder --yes

import json
import requests
import folium
import geocoder

import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

# Part A
### Scrape the Wiki URL and extract postal codes with associated boroughs and neighborhoods

In [2]:
# Inspecting the content, the *table class* is *wikitable sortable* with each *row* identified as **tr** and each *column* as **td**
# Extract the content into a list of tuples

wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_content = requests.get(wiki_url).text
soup = BeautifulSoup(wiki_content, 'lxml')
wiki_table = soup.find('table', {'class': 'wikitable sortable'}).findAll('tr')

data = []
for row in wiki_table:
    tds = row.find_all('td')
    if not tds:
        continue
    (postalcode, borough, neighborhood) = [td.text.strip() for td in tds]
    data.append((postalcode, borough, neighborhood))

In [3]:
# Create dataframe from extracted data
# Drop rows where the Borough is *Not assigned*
# Set Neighborhood name to Borough name is Neighborhood is *Not assigned*

columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(data, columns=columns)

toronto_data = toronto_data[toronto_data.Borough != 'Not assigned'].reset_index(drop=True)

mask = toronto_data.Neighborhood == 'Not assigned'
toronto_data.loc[mask, 'Neighborhood'] = toronto_data.loc[mask, 'Borough']

print('The dataframe has {} boroughs and {} neighborhoods with {} postal codes'.format(len(toronto_data['Borough'].unique()), toronto_data.shape[0], len(toronto_data.PostalCode.unique())))

The dataframe has 11 boroughs and 212 neighborhoods with 103 postal codes


In [4]:
# Group rows with identical PostalCode
# Merge Neighborhoods into one row for each PostalCode

toronto_data = toronto_data.groupby('PostalCode').agg({'Borough': 'first',
                                    'Neighborhood': ', '.join}).reset_index()[columns]

print('Total number of rows in dataframe: {}'.format(toronto_data.shape[0]))
print('Total number of unique PostalCode in dataframe: {}'.format(len(toronto_data.PostalCode.unique())))

Total number of rows in dataframe: 103
Total number of unique PostalCode in dataframe: 103


In [5]:
# Sanity Check: Confirm the dataframe matches the image shown on the assignment page (ignore index value on result)

postalcodes = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']
toronto_data[toronto_data.PostalCode.isin(postalcodes)]

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
5,M1J,Scarborough,Scarborough Village
11,M1R,Scarborough,"Maryvale, Wexford"
17,M2H,North York,Hillcrest Village
35,M4B,East York,"Woodbine Gardens, Parkview Hill"
38,M4G,East York,Leaside
43,M4M,East Toronto,Studio District
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"
57,M5G,Downtown Toronto,Central Bay Street
68,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


# Part B
### Add geographical coordinates to each postal code from Part A

In [6]:
# Attempt to use geocoder to get lat_lng_coords

# def get_lat_lng_coords(row):
#     lat_lng_coords = None
#     while(lat_lng_coords is None):
#         print(row.PostalCode, lat_lng_coords)
#         g = geocoder.google('{}, Toronto, Ontario'.format(row.PostalCode))
#         lat_lng_coords = g.latlng

#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]

#     return latitude, longitude   

# toronto_data['Latitude'], toronto_data['Longitude'] = toronto_data.apply(get_lat_lng_coords, axis=1)

In [7]:
# Get csv file with latitude, longitude data
# Rename *Postal Code* to *PostalCode* to allow merge

!wget -q -O 'geospatial_data.csv' https://cocl.us/Geospatial_data
geospatial_data = pd.read_csv('geospatial_data.csv')
geospatial_data.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [8]:
# Use inner join to merge dataframes

toronto_geospatial_data = pd.merge(toronto_data, geospatial_data, on='PostalCode', how='inner')

In [9]:
# Sanity Check: Confirm the dataframe matches the image shown on the assignment page (ignore index value on result)

postalcodes = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']
toronto_geospatial_data[toronto_geospatial_data.PostalCode.isin(postalcodes)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
38,M4G,East York,Leaside,43.709060,-79.363452
43,M4M,East Toronto,Studio District,43.659526,-79.340923
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
68,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


# Part C
### Cluster neighborhoods

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
def getNearbyVenues(names, latitudes, longitudes):
    venues_list=[]
    drop_neighborhoods = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venue_list = [(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results]
        if len(venue_list) > 0:
            venues_list.append(venue_list)
        else:
            drop_neighborhoods.append(name)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return nearby_venues, drop_neighborhoods

toronto_venues, drop_neighborhoods = getNearbyVenues(names=toronto_geospatial_data['Neighborhood'], latitudes=toronto_geospatial_data['Latitude'], longitudes=toronto_geospatial_data['Longitude'])

In [12]:
# Drop Neighborhoods with no nearby venues data

toronto_data = toronto_data[~toronto_data.Neighborhood.isin(drop_neighborhoods)].reset_index(drop=True)
toronto_geospatial_data = toronto_geospatial_data[~toronto_geospatial_data.Neighborhood.isin(drop_neighborhoods)].reset_index(drop=True)

In [13]:
print('There are {} uniques categories'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} unique neighborhoods'.format(len(toronto_venues['Neighborhood'].unique())))

There are 335 uniques categories
There are 102 unique neighborhoods


In [14]:
# One=Hot Encoding

toronto = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [15]:
# Insert Neighborhood column

toronto['Neighborhood2'] = toronto_venues['Neighborhood']
toronto.drop('Neighborhood', axis=1, inplace=True)
toronto.rename(columns={'Neighborhood2': 'Neighborhood'}, inplace=True)
cols = [toronto.columns[-1]] + list(toronto.columns[:-1])
toronto = toronto[cols]

In [16]:
# Group neighborhoods with the mean of occurrence of each category

toronto_grouped = toronto.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski C

In [17]:
# Get the 20 most common venues in a neighborhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
num_top_venues = 20

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

## K-Means Clustering

In [18]:
# Run k-means clustering

kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_geospatial_data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Clustering Result
## Cluster Map

In [19]:
# Create map of Toronto with markers for clusters

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [20]:
cluster_cols = ['Borough', 'Neighborhood'] + [col for col in toronto_merged.columns if 'Venue' in col]

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, cluster_cols]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
9,Scarborough,"Birch Cliff, Cliffside West",Park,Thai Restaurant,Café,Bank,Diner,Gym,Skating Rink,Gym Pool,General Entertainment,Restaurant,Dessert Shop,College Stadium,Fast Food Restaurant,Falafel Restaurant,Farm,Farmers Market,Field,Filipino Restaurant,Fireworks Store,Exhibit
18,North York,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Grocery Store,Intersection,Shopping Mall,Skating Rink,Skate Park,Fast Food Restaurant,Farm,Falafel Restaurant,Farmers Market,Field,Filipino Restaurant,Exhibit,Fireworks Store,Event Space,Fish & Chips Shop,Ethiopian Restaurant
22,North York,York Mills West,Park,Tennis Court,Restaurant,Intersection,Diner,Bank,Golf Course,French Restaurant,Dog Run,Coffee Shop,Grocery Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Zoo,Exhibit,Eastern European Restaurant,Falafel Restaurant,Farm
30,North York,Downsview West,Park,Coffee Shop,Pizza Place,Shopping Mall,Vietnamese Restaurant,Moving Target,Grocery Store,Bank,Exhibit,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Zoo,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
79,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",Furniture / Home Store,Grocery Store,Convenience Store,Turkish Restaurant,Fast Food Restaurant,Check Cashing Service,Sandwich Place,Park,Dessert Shop,Italian Restaurant,Restaurant,Discount Store,Wine Shop,Playground,Hardware Store,Deli / Bodega,Event Space,Electronics Store,Cosmetics Shop,Elementary School
87,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",Park,Indian Restaurant,Skating Rink,Italian Restaurant,Liquor Store,Restaurant,Grocery Store,Bakery,Dessert Shop,Café,Pharmacy,Pub,American Restaurant,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Fried Chicken Joint,Exhibit,Event Space,Ethiopian Restaurant
90,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",Park,Italian Restaurant,Eastern European Restaurant,Shopping Mall,Ice Cream Shop,Falafel Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Zoo,Dumpling Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop


## Cluster 2

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, cluster_cols]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Scarborough,"Rouge, Malvern",Fast Food Restaurant,Coffee Shop,Spa,Fruit & Vegetable Store,Chinese Restaurant,Sandwich Place,Caribbean Restaurant,Paper / Office Supplies Store,Park,Gym,Arts & Crafts Store,Bus Station,Hobby Shop,Falafel Restaurant,Exhibit,Eastern European Restaurant,Farm,Farmers Market,Event Space,Field
2,Scarborough,"Guildwood, Morningside, West Hill",Pizza Place,Fast Food Restaurant,Coffee Shop,Food & Drink Shop,Greek Restaurant,Smoothie Shop,Sports Bar,Fried Chicken Joint,Liquor Store,Supermarket,Beer Store,Sandwich Place,Discount Store,Bank,Burger Joint,Pharmacy,Farmers Market,Curling Ice,Exhibit,Falafel Restaurant
3,Scarborough,Woburn,Coffee Shop,Chinese Restaurant,Electronics Store,Park,Fast Food Restaurant,Indian Restaurant,Pharmacy,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farm,Falafel Restaurant,Dumpling Restaurant,Farmers Market,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop
4,Scarborough,Cedarbrae,Pharmacy,Bakery,Coffee Shop,Indian Restaurant,Pizza Place,Grocery Store,Burger Joint,Fast Food Restaurant,Sporting Goods Shop,Fried Chicken Joint,Caribbean Restaurant,Music Store,Chinese Restaurant,Bus Line,Thai Restaurant,Rental Car Location,Bank,Athletics & Sports,Yoga Studio,Hakka Restaurant
5,Scarborough,Scarborough Village,Fast Food Restaurant,Convenience Store,Coffee Shop,Bowling Alley,Japanese Restaurant,Train Station,Restaurant,Sandwich Place,Pizza Place,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Curling Ice,Eastern European Restaurant,Coworking Space
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Coffee Shop,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Grocery Store,Pizza Place,Rental Car Location,Metro Station,Light Rail Station,Sandwich Place,Bank,Bus Line,Train Station,Burger Joint,Asian Restaurant,Department Store,Hockey Arena,Hobby Shop,Farmers Market
7,Scarborough,"Clairlea, Golden Mile, Oakridge",Coffee Shop,Bus Line,Convenience Store,Intersection,Diner,Fast Food Restaurant,Bakery,Metro Station,General Entertainment,Beer Store,Mexican Restaurant,Business Service,Soccer Field,Bus Station,Sandwich Place,Park,Pub,Bank,Pizza Place,Gym
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",Pizza Place,Beach,Fast Food Restaurant,Wings Joint,Burger Joint,Sports Bar,Cajun / Creole Restaurant,Furniture / Home Store,Park,Zoo,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Farm,Electronics Store,Field,Filipino Restaurant
11,Scarborough,"Maryvale, Wexford",Pizza Place,Grocery Store,Middle Eastern Restaurant,Convenience Store,Korean Restaurant,Badminton Court,Bakery,Restaurant,Bar,Smoke Shop,Supermarket,Coffee Shop,Seafood Restaurant,Breakfast Spot,Burger Joint,Soccer Field,Flea Market,Intersection,Fish Market,Indian Restaurant
12,Scarborough,Agincourt,Chinese Restaurant,Caribbean Restaurant,Supermarket,Bakery,Coffee Shop,Shopping Mall,Sandwich Place,Sri Lankan Restaurant,Breakfast Spot,Bubble Tea Shop,Malay Restaurant,Skating Rink,Mediterranean Restaurant,Shanghai Restaurant,Seafood Restaurant,Lounge,Cantonese Restaurant,Sushi Restaurant,Restaurant,Motorcycle Shop


## Cluster 3

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, cluster_cols]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
19,North York,"Silver Hills, York Mills",Park,Pool,Zoo,Farm,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Dumpling Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market


## Cluster 4

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, cluster_cols]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
31,North York,Downsview Central,Vietnamese Restaurant,Restaurant,Baseball Field,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Zoo,Eastern European Restaurant,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market


## Cluster 5

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, cluster_cols]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",Breakfast Spot,Burger Joint,Playground,Italian Restaurant,Zoo,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Field,Fast Food Restaurant,Electronics Store,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market
10,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",Electronics Store,Asian Restaurant,Coffee Shop,Burger Joint,Indian Restaurant,Bakery,Chinese Restaurant,Automotive Shop,Fast Food Restaurant,Furniture / Home Store,Beer Store,Clothing Store,Intersection,Fried Chicken Joint,Gaming Cafe,Sandwich Place,Bank,Latin American Restaurant,Light Rail Station,Italian Restaurant
17,North York,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Bakery,Japanese Restaurant,Sandwich Place,Beer Store,Shopping Mall,Supermarket,Restaurant,Movie Theater,Juice Bar,Electronics Store,Bank,Tea Room,Caribbean Restaurant,Sporting Goods Shop,Candy Store,Salon / Barbershop,Food Court,Liquor Store
20,North York,"Newtonbrook, Willowdale",Korean Restaurant,Café,Pizza Place,Coffee Shop,Shopping Mall,Supermarket,Fried Chicken Joint,Middle Eastern Restaurant,Bus Station,Sandwich Place,Japanese Restaurant,Electronics Store,Dessert Shop,Park,Ramen Restaurant,Convenience Store,Bank,Hot Dog Joint,Indian Restaurant,Halal Restaurant
21,North York,Willowdale South,Japanese Restaurant,Ramen Restaurant,Coffee Shop,Korean Restaurant,Pizza Place,Bubble Tea Shop,Café,Sandwich Place,Sushi Restaurant,Dessert Shop,Fast Food Restaurant,Restaurant,Italian Restaurant,Pharmacy,Vietnamese Restaurant,Fried Chicken Joint,Chinese Restaurant,Hotel,Indonesian Restaurant,Beer Store
25,North York,Don Mills North,Japanese Restaurant,Pizza Place,Coffee Shop,Burger Joint,Salad Place,Thai Restaurant,Bank,Baseball Field,Restaurant,Supermarket,Bar,Greek Restaurant,Fast Food Restaurant,Breakfast Spot,Caribbean Restaurant,Café,Cafeteria,Liquor Store,Mobile Phone Shop,Ice Cream Shop
26,North York,"Flemingdon Park, Don Mills South",Restaurant,Beer Store,American Restaurant,Grocery Store,Gym,Asian Restaurant,Japanese Restaurant,Coffee Shop,Sporting Goods Shop,Chinese Restaurant,Bike Shop,Sandwich Place,Electronics Store,Supermarket,Fast Food Restaurant,Burger Joint,Café,Middle Eastern Restaurant,Sushi Restaurant,General Entertainment
29,North York,"CFB Toronto, Downsview East",Coffee Shop,Electronics Store,Turkish Restaurant,Pizza Place,Café,Middle Eastern Restaurant,Latin American Restaurant,Italian Restaurant,Liquor Store,Park,Gym,Sandwich Place,Vietnamese Restaurant,Airport,Soccer Field,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
33,North York,Victoria Village,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Pizza Place,Golf Course,Men's Store,Park,Lounge,Café,Athletics & Sports,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Electronics Store,Farmers Market,Fast Food Restaurant
36,East Toronto,The Beaches,Pub,Coffee Shop,Beach,Pizza Place,Breakfast Spot,Bar,Bakery,Tea Room,Burger Joint,Park,Sandwich Place,Caribbean Restaurant,Japanese Restaurant,Kids Store,Sporting Goods Shop,Juice Bar,Shopping Mall,Jewelry Store,Jazz Club,Bookstore
